In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np

In [2]:
import os
from tqdm import tqdm

In [3]:
import sys
sys.path.append('../datasets/')

In [4]:
from dataset_utils import int64_feature, bytes_feature

# convert to tfrecord

In [5]:
## fetch all property of one image
## make every single value as a property
## it means multi-dimensions value will be
## decomposed into several single values
# read image data
# get image height
# get image width
# get image depth
# get image label

In [6]:
## compose features
## make example
## create tfrecord_writer
# write example to file

In [7]:
dir_names = ['../data/0/', '../data/90/', '../data/180/', '../data/270/']

In [8]:
dir_to_label = dict(zip(dir_names, range(1, len(dir_names)+1)))

In [9]:
def example_gen():
    for dir_name in dir_names:
        filenames = os.listdir(dir_name)
        filenames = [n for n in filenames if os.path.splitext(n)[1].lower() in ['.jpg', '.jpeg']]
        for filename in tqdm(filenames):
            path = os.path.join(dir_name, filename)
            image_data = tf.gfile.FastGFile(path, 'r').read()
            yield tf.train.Example(features=tf.train.Features(feature={
                'image/encode': bytes_feature(image_data),
                'image/format': bytes_feature(b'JPEG'),
                'image/label': int64_feature(dir_to_label[dir_name])
            }))

In [14]:
a = example_gen()

In [15]:
fidx = 0
SAMPLE_PER_FILES = 200
b_quit = False
while True:
    tf_filename = "orient_train_{:03d}.tfrecord".format(fidx)
    with tf.python_io.TFRecordWriter(tf_filename) as tfrecord_writer:
        for j in range(SAMPLE_PER_FILES):
            try:
                example = next(a)
                tfrecord_writer.write(example.SerializeToString())
            except StopIteration:
                b_quit = True
            finally:
                if b_quit: break
    if b_quit: break
    fidx += 1

100%|██████████| 166/166 [00:00<00:00, 303.86it/s]


In [16]:
! open .